In [ ]:
!pip install ax-platform

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.3/530.3 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.7/730.7 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.7/156.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.0 MB/s eta 0:00:00


In [ ]:
!pip install snnTorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.7/104.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
from ax.service.managed_loop import optimize
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.utils.notebook.plotting import render, init_notebook_plotting

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchsummary import summary
import numpy as np

In [ ]:
import snntorch as snn

In [ ]:
torch.manual_seed(12345)
dtype = torch.float
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Load and Split MNIST

Defining transform before downloading MNIST in a local folder

In [ ]:
batch_size = 128
data_path='/data/mnist'


transform = transforms.Compose([
            transforms.Resize((28, 28)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 98987903.19it/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 77156492.88it/s]

Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 24940735.60it/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 9948056.80it/s]

Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



In [ ]:
mnist_test

Dataset MNIST
    Number of datapoints: 10000
    Root location: /data/mnist
    Split: Test
    StandardTransform
Transform: Compose(
               Resize(size=(28, 28), interpolation=bilinear, max_size=None, antialias=warn)
               Grayscale(num_output_channels=1)
               ToTensor()
               Normalize(mean=(0,), std=(1,))
           )

# Single-Objective SNN Optimization

In [ ]:
class SNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        self.lif2 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        
        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        for step in range(num_steps):
            cur1 = self.fc1(x)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)

In [ ]:
# Network Architecture
num_inputs = 28*28
num_hidden = 1000
num_outputs = 10

# Temporal Dynamics
num_epochs = 1
num_steps = 25
beta = 0.95

In [ ]:
def train_evaluate(parameterization):
    model = SNN()
    model.to(device)

    loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),
                           lr=parameterization.get("lr", 0.001),
                           betas=(parameterization.get("beta1", 0.9), 0.999))
    
    def print_batch_accuracy(data, targets, train=False):
        output, _ = model(data.view(batch_size, -1))
        _, idx = output.sum(dim=0).max(1)
        acc = np.mean((targets == idx).detach().cpu().numpy())

        if train:
            print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
        else:
            print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

    loss_hist = []
    test_loss_hist = []
    accuracy = 0
    counter = 0

    # Outer training loop
    for epoch in range(num_epochs):
        iter_counter = 0
        train_batch = iter(train_loader)

        # Minibatch training loop
        for data, targets in train_batch:
            data = data.to(device)
            targets = targets.to(device)

            # forward pass
            model.train()
            spk_rec, mem_rec = model(data.view(batch_size, -1))

            # initialize the loss & sum over time
            loss_val = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
                loss_val += loss(mem_rec[step], targets)

            # Gradient calculation + weight update
            optimizer.zero_grad()
            loss_val.backward()
            optimizer.step()

            # Store loss history for future plotting
            loss_hist.append(loss_val.item())

            # Test set
            with torch.no_grad():
                model.eval()
                test_data, test_targets = next(iter(test_loader))
                test_data = test_data.to(device)
                test_targets = test_targets.to(device)

                # Test set forward pass
                test_spk, test_mem = model(test_data.view(batch_size, -1))
                _, idx = test_spk.sum(dim=0).max(1)
                
                actual_acc = np.mean((test_targets == idx).detach().cpu().numpy())

                if actual_acc > accuracy:
                  accuracy = actual_acc

                # Test set loss
                test_loss = torch.zeros((1), dtype=dtype, device=device)
                for step in range(num_steps):
                    test_loss += loss(test_mem[step], test_targets)
                test_loss_hist.append(test_loss.item())

                # Print train/test loss/accuracy
                """
                if counter % 50 == 0:
                    print(f"Epoch {epoch}, Iteration {iter_counter}")
                    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
                    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
                    print_batch_accuracy(data, targets, train=True)
                    print_batch_accuracy(test_data, test_targets, train=False)
                    print("\n")
                """
                counter += 1
                iter_counter +=1
    return accuracy

In [ ]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.4], "log_scale": True, "value_type": 'float'},
        {"name": "beta1", "type": "range", "bounds": [0.5, 0.999], "value_type": 'float'},
    ],
    evaluation_function=train_evaluate,
    objective_name='accuracy',
    total_trials = 20
)

[INFO 05-09 14:30:58] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='lr', parameter_type=FLOAT, range=[1e-06, 0.4], log_scale=True), RangeParameter(name='beta1', parameter_type=FLOAT, range=[0.5, 0.999])], parameter_constraints=[]).
[INFO 05-09 14:30:58] ax.modelbridge.dispatch_utils: Using Models.GPEI since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 05-09 14:30:58] ax.modelbridge.dispatch_utils: Calculating the number of remaining initialization trials based on num_initialization_trials=None max_initialization_trials=None num_tunable_parameters=2 num_trials=None use_batch_trials=False
[INFO 05-09 14:30:58] ax.modelbridge.dispatch_utils: calculated num_initialization_trials=5
[INFO 05-09 14:30:58] ax.modelbridge.dispatch_utils: num_completed_initialization_trials=0 num_remaining_initialization_trials=5
[INFO 05-09 14:30:58] ax.modelbridge.dispatch_utils: Using Bayesian O

In [ ]:
best_parameters

{'lr': 0.0024901629078190285, 'beta1': 0.8972629127828733}

In [ ]:
values

({'accuracy': 0.9943091213245984},
 {'accuracy': {'accuracy': 2.0439282400765623e-05}})

In [ ]:
experiment

Experiment(None)

In [ ]:
model

# Multi-Objective SNN Optimization

In [ ]:
from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties

import torch

from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.plot.pareto_utils import compute_posterior_pareto_frontier
from ax.plot.pareto_frontier import plot_pareto_frontier

import time

[WARNING 05-22 13:47:49] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


In [ ]:
ax_client = AxClient()
ax_client.create_experiment(
    name="snn_experiment",
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.4], "log_scale": True, "value_type": 'float'},
        {"name": "beta1", "type": "range", "bounds": [0.5, 0.999], "value_type": 'float'},
    ],
    objectives={
        "accuracy": ObjectiveProperties(minimize=False), 
        "time": ObjectiveProperties(minimize=True)
    },
    overwrite_existing_experiment=True,
    is_test=True,
)

[INFO 05-22 14:21:23] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 05-22 14:21:23] ax.service.utils.instantiation: Due to non-specification, we will use the heuristic for selecting objective thresholds.
[INFO 05-22 14:21:23] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='lr', parameter_type=FLOAT, range=[1e-06, 0.4], log_scale=True), RangeParameter(name='beta1', parameter_type=FLOAT, range=[0.5, 0.999])], parameter_constraints=[]).
[INFO 05-22 14:21:23] ax.core.experiment: The is_test flag has been set to True. This flag is meant purely for development and integration testing purposes. If you are running a live experiment, please set this flag to False
[INFO 05-22 14:21:23] ax.modelbridge.dispatch_utils: Using Models.MOO since there are more ordered parameters than there are 

In [ ]:
def train_evaluate_multi(parameterization):
    model = SNN()
    model.to(device)

    loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),
                           lr=parameterization.get("lr", 0.001),
                           betas=(parameterization.get("beta1", 0.9), 0.999))
    
    def print_batch_accuracy(data, targets, train=False):
        output, _ = model(data.view(batch_size, -1))
        _, idx = output.sum(dim=0).max(1)
        acc = np.mean((targets == idx).detach().cpu().numpy())

        if train:
            print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
        else:
            print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

    loss_hist = []
    test_loss_hist = []
    accuracy = 0
    counter = 0
    eval_time = []
    cuda_eval_time = []

    """
    # Correct way to measure elapsed time during evaluations
    
    # INIT LOGGERS
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    repetitions = 300
    timings=np.zeros((repetitions,1))
    #GPU-WARM-UP
    for _ in range(10):
        _ = model(dummy_input)
    # MEASURE PERFORMANCE
    with torch.no_grad():
        for rep in range(repetitions):
            starter.record()
            _ = model(dummy_input)
            ender.record()
            # WAIT FOR GPU SYNC
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)
            timings[rep] = curr_time
    mean_syn = np.sum(timings) / repetitions
    std_syn = np.std(timings)
    print(mean_syn)
    """

    

    # Outer training loop
    for epoch in range(num_epochs):
        iter_counter = 0
        train_batch = iter(train_loader)

        # Minibatch training loop
        for data, targets in train_batch:
            data = data.to(device)
            targets = targets.to(device)

            # forward pass
            model.train()
            spk_rec, mem_rec = model(data.view(batch_size, -1))

            # initialize the loss & sum over time
            loss_val = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
                loss_val += loss(mem_rec[step], targets)

            # Gradient calculation + weight update
            optimizer.zero_grad()
            loss_val.backward()
            optimizer.step()

            # Store loss history for future plotting
            loss_hist.append(loss_val.item())

            # Test set
            with torch.no_grad():
                model.eval()
                test_data, test_targets = next(iter(test_loader))
                test_data = test_data.to(device)
                test_targets = test_targets.to(device)

                if torch.cuda.is_available():
                  starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
                  #GPU-WARM-UP
                  for _ in range(5):
                      _ = model(test_data.view(batch_size, -1))
                  starter.record()

                else:
                  starter = time.time()
                # Test set forward pass
                test_spk, test_mem = model(test_data.view(batch_size, -1))
                if torch.cuda.is_available():
                  ender.record()
                  # WAIT FOR GPU SYNC
                  torch.cuda.synchronize()
                  curr_time = starter.elapsed_time(ender)
                  cuda_eval_time.append(curr_time)
                else:
                  ender = time.time()
                  elapsed_time = ender - starter
                  eval_time.append(elapsed_time)

                _, idx = test_spk.sum(dim=0).max(1)
                
                actual_acc = np.mean((test_targets == idx).detach().cpu().numpy())

                if actual_acc > accuracy:
                  accuracy = actual_acc

                # Test set loss
                test_loss = torch.zeros((1), dtype=dtype, device=device)
                for step in range(num_steps):
                    test_loss += loss(test_mem[step], test_targets)
                test_loss_hist.append(test_loss.item())

                # Print train/test loss/accuracy
                """
                if counter % 50 == 0:
                    print(f"Epoch {epoch}, Iteration {iter_counter}")
                    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
                    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
                    print_batch_accuracy(data, targets, train=True)
                    print_batch_accuracy(test_data, test_targets, train=False)
                    print("\n")
                """
                counter += 1
                iter_counter +=1
    
    if torch.cuda.is_available():
      avg_time = np.mean(cuda_eval_time)
    else:
      avg_time = np.mean(eval_time)

    return {"accuracy": accuracy, "time": avg_time}

In [ ]:
for i in range(10):
    parameters, trial_index = ax_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    ax_client.complete_trial(trial_index=trial_index, raw_data=train_evaluate_multi(parameters))

[INFO 05-22 14:21:29] ax.service.ax_client: Generated new trial 0 with parameters {'lr': 0.000781, 'beta1': 0.970662}.
[INFO 05-22 14:26:01] ax.service.ax_client: Completed trial 0 with data: {'accuracy': (0.992188, None), 'time': (0.160135, None)}.
[INFO 05-22 14:26:01] ax.service.ax_client: Generated new trial 1 with parameters {'lr': 0.009295, 'beta1': 0.920814}.
[INFO 05-22 14:30:33] ax.service.ax_client: Completed trial 1 with data: {'accuracy': (1.0, None), 'time': (0.158482, None)}.
[INFO 05-22 14:30:33] ax.service.ax_client: Generated new trial 2 with parameters {'lr': 0.093229, 'beta1': 0.623166}.
[INFO 05-22 14:38:52] ax.service.ax_client: Completed trial 2 with data: {'accuracy': (0.25, None), 'time': (0.16159, None)}.
[INFO 05-22 14:38:52] ax.service.ax_client: Generated new trial 3 with parameters {'lr': 4e-06, 'beta1': 0.935298}.
[INFO 05-22 14:43:27] ax.service.ax_client: Completed trial 3 with data: {'accuracy': (0.828125, None), 'time': (0.160559, None)}.
[INFO 05-22 1

KeyboardInterrupt: ignored

In [ ]:
objectives = ax_client.experiment.optimization_config.objective.objectives
frontier = compute_posterior_pareto_frontier(
    experiment=ax_client.experiment,
    data=ax_client.experiment.fetch_data(),
    primary_objective=objectives[1].metric,
    secondary_objective=objectives[0].metric,
    absolute_metrics=["accuracy", "time"],
    num_points=100,
)
render(plot_pareto_frontier(frontier, CI_level=0.90)) 